In [19]:
import pyspark
from pyspark.sql import SparkSession

sc = pyspark.SparkContext()
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-153f8b0be037>:4 

In [2]:
user_df = spark.read.format('csv').option('header', 'true').load('users.csv')
user_df.count()

100

In [3]:
order_df = spark.read.format('csv').option('header', 'true').load('orders.csv')
order_df.count()

1000000

In [15]:
#spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '-1')
userOrderDF = order_df.join(user_df.hint('broadcast'), user_df.user_id == order_df.user_id)

In [16]:
userOrderDF.show()

+--------+---+----------+-------+-------+---------+----------+
|order_id|qty|product_id|user_id|user_id|user_name|user_state|
+--------+---+----------+-------+-------+---------+----------+
|       0| 27|         4|      3|      0|   user_0|        CO|
|       1| 82|         2|     33|      1|   user_1|        AZ|
|       2| 46|         0|     35|      2|   user_2|        MI|
|       3| 61|         1|     88|      3|   user_3|        CO|
|       4| 61|         0|     44|      4|   user_4|        CA|
|       5| 87|         3|     76|      5|   user_5|        WA|
|       6| 24|         0|     65|      6|   user_6|        CO|
|       7| 42|         4|     99|      7|   user_7|        NY|
|       8| 14|         4|     23|      8|   user_8|        NY|
|       9| 16|         0|     79|      9|   user_9|        CO|
|      10| 23|         4|     69|     10|  user_10|        NY|
|      11|  3|         2|     11|     11|  user_11|        CO|
|      12| 98|         2|     41|     12|  user_12|    

In [17]:
userOrderDF.explain()

== Physical Plan ==
*(2) BroadcastHashJoin [order_id#46], [user_id#16], Inner, BuildRight
:- *(2) Project [order_id#46, qty#47, product_id#48, user_id#49]
:  +- *(2) Filter isnotnull(order_id#46)
:     +- FileScan csv [order_id#46,qty#47,product_id#48,user_id#49] Batched: false, DataFilters: [isnotnull(order_id#46)], Format: CSV, Location: InMemoryFileIndex[file:/home/andras/ipython_spark/spark_training_baseline/06_joins/orders.csv], PartitionFilters: [], PushedFilters: [IsNotNull(order_id)], ReadSchema: struct<order_id:string,qty:string,product_id:string,user_id:string>
+- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, true])), [id=#400]
   +- *(1) Project [user_id#16, user_name#17, user_state#18]
      +- *(1) Filter isnotnull(user_id#16)
         +- FileScan csv [user_id#16,user_name#17,user_state#18] Batched: false, DataFilters: [isnotnull(user_id#16)], Format: CSV, Location: InMemoryFileIndex[file:/home/andras/ipython_spark/spark_training_baseline/06_joins/use

In [20]:
user_df.count()

100

In [22]:
user_df.limit(10)

DataFrame[user_id: string, user_name: string, user_state: string]

In [25]:
userOrderDF = order_df.join(user_df.hint('broadcast'), ['user_id'], 'inner')
userOrderDF.show()

+-------+--------+---+----------+---------+----------+
|user_id|order_id|qty|product_id|user_name|user_state|
+-------+--------+---+----------+---------+----------+
|      3|       0| 27|         4|   user_3|        CO|
|     33|       1| 82|         2|  user_33|        AZ|
|     35|       2| 46|         0|  user_35|        WA|
|     88|       3| 61|         1|  user_88|        MI|
|     44|       4| 61|         0|  user_44|        MI|
|     76|       5| 87|         3|  user_76|        AZ|
|     65|       6| 24|         0|  user_65|        NY|
|     99|       7| 42|         4|  user_99|        CA|
|     23|       8| 14|         4|  user_23|        CA|
|     79|       9| 16|         0|  user_79|        CA|
|     69|      10| 23|         4|  user_69|        CA|
|     11|      11|  3|         2|  user_11|        CO|
|     41|      12| 98|         2|  user_41|        CO|
|     78|      13| 29|         2|  user_78|        MI|
|     28|      14| 40|         1|  user_28|        NY|
|     30| 

In [29]:
userOrderDF.rdd.getNumPartitions()

4